In [1]:
"""
Working on a big refactor here...
moving towards vrAnalysis2

Objectives:
- Build dataclass structure for imaging and behavior sessions that I can operate on
across datatypes - I want to run the same code on my data and external data
- Simplify analyses by building a set of functions that can be called on these dataclasses
instead of extending analysis classes for each kind of analysis
- Rewrite standard scripts to do analyses with this new refactor


I should think about the key requirements for analyzing behavior and imaging data for
vr navigation and structure around that. Things that I do:

Basic data handling:
- Load spiking data
- Load behavior data (position, speed, etc)
- Load ROI data

Processing:
- Make a spkmap
- Process spkmaps
- Filter and sort ROIs by tracked (or other criteria, like plane_idx...)
"""


"""
Ideas:
An analysis object which can be loaded that isn't necessarily connect to any data. You can pass in a list
of data objects to it and it will run the analysis on all of them. This way I can run the same analysis on
my data and external data, compare within a mouse, across mice, etc.

There can be a "flag" for same mice with tracker, in which case it can perform special operations like aligning
ROIs across sessions by tracked ROI etc. 
"""

"""
Updates / Work in progress:
- I've got the b2session working. I think it's good. There might be more to add or reorganize, but essentially it's
  ready to go because it can find the onedata, load spks in a clever way, and has a few other tools for loading. 
- Now I think I can just keep working on everything that comes after this... e.g. on the analysis class side of things
  which might need a lot of refactoring (or maybe not???)
- The registration stuff will be nice to refactor to this method, but for now I can hack it and just use both I guess?
"""

print()

In [19]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt

from vrAnalysis2.helpers import Timer
from vrAnalysis2.sessions import create_b2session
from vrAnalysis2.processors.spkmaps import SpkmapProcessor, SpkmapParams
mouse_name = "ATL027"
date = "2023-07-27"
session_id = "701"
spks_type = "significant"

session = create_b2session(mouse_name, date, session_id, dict(spks_type=spks_type))
spkmap_processor = SpkmapProcessor(session)

In [2]:
spkmap_processor.cache_directory()

WindowsPath('D:/localData/ATL027/2023-07-27/701/spkmaps')

In [4]:
with Timer():
    maps = spkmap_processor.get_env_maps(use_session_filters=True)
    maps.pop_nan_positions()

Elapsed time: 0.5760 seconds


In [ ]:
with Timer():
    spkmap_processor.get_reliability()

Elapsed time: 0.9062 seconds


In [ ]:
with Timer():
    reliability = spkmap_processor.get_reliability(force_recompute=True, use_session_filters=True)

Elapsed time: 2.9719 seconds


In [42]:
with Timer():
    reliability_cache = spkmap_processor.get_reliability(use_session_filters=True)

Elapsed time: 0.0272 seconds


In [37]:
reliability.values.shape

(2, 16488)

In [ ]:
reliability_cache.values.shape

(2, 16488)

In [44]:
from vrAnalysis2.database import get_database
sessiondb = get_database("vrSessions")
ises = sessiondb.iter_sessions(imaging=True)

In [45]:
print(ises[0])

B2Session(mouse_name='ATL012', date='2023-01-20', session_id='702')


In [3]:
# TODO LIST
# - Need to add the auxiliary methods like reliability and frame behavior etc

In [1]:
# Speed up again....
# Implement some extra checks for spkmapProcessor to make sure get_env_maps can be cached or loaded from cache (e.g. if it's standard popnan, no average, etc etc)
# Then same for reliability
# And set it up so that I can still filter etc and but load the full precomputed reliability metric 